In [1]:
#Dependencies
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from pathlib import Path
import matplotlib.pyplot as plt
from spot_secrets import client_id, client_secret

In [3]:
#read csv as dataframe
us_top_200 = pd.read_csv('Output/USA_top200.csv')

#up_top_200 confirmation
us_top_200.head()

,title,rank,date,artist,url,region,chart,trend,streams
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.0
1,Fake Love,2,2017-01-01,Drake,https://open.spotify.com/track/343YBumqHu19cGo...,United States,top200,SAME_POSITION,1180074.0
2,Starboy,3,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,United States,top200,SAME_POSITION,1064351.0
3,Closer,4,2017-01-01,"The Chainsmokers, Halsey",https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,United States,top200,SAME_POSITION,1010492.0
4,Black Beatles,5,2017-01-01,"Rae Sremmurd, Gucci Mane",https://open.spotify.com/track/6fujklziTHa8uoM...,United States,top200,SAME_POSITION,874289.0


In [4]:
#Cleans up data by removing duplicates, keeping 1st date.
title_by_month_df = us_top_200.groupby('title').first().reset_index()
title_by_month_df.head(100)

,title,rank,date,artist,url,region,chart,trend,streams
0,!,13,2019-08-09,Trippie Redd,https://open.spotify.com/track/5a1ofLoPiAn5xxf...,United States,top200,NEW_ENTRY,808145.0
1,"""42"" - From SR3MM",130,2018-05-04,"Rae Sremmurd, Swae Lee, Slim Jxmmi",https://open.spotify.com/track/5lf91lPnGKtIqsg...,United States,top200,NEW_ENTRY,297731.0
2,#PROUDCATOWNERREMIX,89,2019-08-23,"XXXTENTACION, Rico Nasty",https://open.spotify.com/track/5dkZ2DrcPJrqwlR...,United States,top200,NEW_ENTRY,361205.0
3,$$$ - with Matt Ox,147,2018-04-01,XXXTENTACION,https://open.spotify.com/track/65u1dHQyQyE4y4a...,United States,top200,MOVE_UP,198304.0
4,'Til You Can't,153,2021-10-08,Cody Johnson,https://open.spotify.com/track/4k3lPl8YTKuY8c1...,United States,top200,NEW_ENTRY,249850.0
...,...,...,...,...,...,...,...,...,...
95,2040,33,2021-06-04,"Lil Baby, Lil Durk",https://open.spotify.com/track/2ELp0EL65yitpfJ...,United States,top200,NEW_ENTRY,583200.0
96,2055,125,2021-07-01,Sleepy Hallow,https://open.spotify.com/track/4XvcHTUfIlWfyJT...,United States,top200,MOVE_UP,266434.0
97,21,188,2020-06-01,Polo G,https://open.spotify.com/track/3JKgcAa7r07ocVW...,United States,top200,MOVE_UP,200551.0
98,21C/Delta,119,2020-12-11,Jack Harlow,https://open.spotify.com/track/54FJWt8YIA1IHjm...,United States,top200,NEW_ENTRY,363120.0


In [8]:
#turns date column data type from object to datetime64[ns]
title_by_month_df['date'] = pd.to_datetime(title_by_month_df['date'])
title_by_month_df.dtypes

title              object
rank                int64
date       datetime64[ns]
artist             object
url                object
region             object
chart              object
trend              object
streams           float64
dtype: object

In [15]:
#check for duplicates in titles, if sum method brings up 0, means no dupes.
dupes = title_by_month_df.duplicated(subset=['title'])
print(dupes.sum())

#just to count rows left after dupe removal
print(f"{dupes.count()} rows left after dupe purge")

0
7543 rows left after dupe purge


In [16]:
#grab Spotify Track Id from URL and add in a new column named 'Spotify Track Id'

title_by_month_df['Spotify Track Id'] = title_by_month_df['url'].str.split('/').str[-1]
title_by_month_df.head(100)

,title,rank,date,artist,url,region,chart,trend,streams,Spotify Track Id
0,!,13,2019-08-09,Trippie Redd,https://open.spotify.com/track/5a1ofLoPiAn5xxf...,United States,top200,NEW_ENTRY,808145.0,5a1ofLoPiAn5xxf5UE6472
1,"""42"" - From SR3MM",130,2018-05-04,"Rae Sremmurd, Swae Lee, Slim Jxmmi",https://open.spotify.com/track/5lf91lPnGKtIqsg...,United States,top200,NEW_ENTRY,297731.0,5lf91lPnGKtIqsgmG1z8Ip
2,#PROUDCATOWNERREMIX,89,2019-08-23,"XXXTENTACION, Rico Nasty",https://open.spotify.com/track/5dkZ2DrcPJrqwlR...,United States,top200,NEW_ENTRY,361205.0,5dkZ2DrcPJrqwlRQe6Q35e
3,$$$ - with Matt Ox,147,2018-04-01,XXXTENTACION,https://open.spotify.com/track/65u1dHQyQyE4y4a...,United States,top200,MOVE_UP,198304.0,65u1dHQyQyE4y4aN2eDmMF
4,'Til You Can't,153,2021-10-08,Cody Johnson,https://open.spotify.com/track/4k3lPl8YTKuY8c1...,United States,top200,NEW_ENTRY,249850.0,4k3lPl8YTKuY8c1HelVnm3
...,...,...,...,...,...,...,...,...,...,...
95,2040,33,2021-06-04,"Lil Baby, Lil Durk",https://open.spotify.com/track/2ELp0EL65yitpfJ...,United States,top200,NEW_ENTRY,583200.0,2ELp0EL65yitpfJYwc5983
96,2055,125,2021-07-01,Sleepy Hallow,https://open.spotify.com/track/4XvcHTUfIlWfyJT...,United States,top200,MOVE_UP,266434.0,4XvcHTUfIlWfyJTRG0aqlo
97,21,188,2020-06-01,Polo G,https://open.spotify.com/track/3JKgcAa7r07ocVW...,United States,top200,MOVE_UP,200551.0,3JKgcAa7r07ocVWcV8bS0H
98,21C/Delta,119,2020-12-11,Jack Harlow,https://open.spotify.com/track/54FJWt8YIA1IHjm...,United States,top200,NEW_ENTRY,363120.0,54FJWt8YIA1IHjmUxuDxwi


In [17]:
#Rename df to "top200"
top200 = title_by_month_df

In [20]:
#FOR REAL DATA - **Taken from Joseph's "spotify_data" notebook.

# Authenticate with the Spotify API using client credentials flow
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Load the Spotify Track Id column from the dataframe
track_ids = top200['Spotify Track Id']

# Create empty lists to store the audio feature data
danceability = []
energy = []
key = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []
duration_ms = []

# Iterate through each track id in the column and make a request to the Spotify API for its audio features
for track_id in track_ids:
    audio_features = sp.audio_features(track_id)[0]
    danceability.append(audio_features['danceability'])
    energy.append(audio_features['energy'])
    key.append(audio_features['key'])
    loudness.append(audio_features['loudness'])
    mode.append(audio_features['mode'])
    speechiness.append(audio_features['speechiness'])
    acousticness.append(audio_features['acousticness'])
    instrumentalness.append(audio_features['instrumentalness'])
    liveness.append(audio_features['liveness'])
    valence.append(audio_features['valence'])
    tempo.append(audio_features['tempo'])
    duration_ms.append(audio_features['duration_ms'])

# Add the audio feature data to the dataframe as new columns
top200['danceability'] = danceability
top200['energy'] = energy
top200['key'] = key
top200['loudness'] = loudness
top200['mode'] = mode
top200['speechiness'] = speechiness
top200['acousticness'] = acousticness
top200['instrumentalness'] = instrumentalness
top200['liveness'] = liveness
top200['valence'] = valence
top200['tempo'] = tempo
top200['duration_ms'] = duration_ms

# Print the updated dataframe
top200.head()

,title,rank,date,artist,url,region,chart,trend,streams,Spotify Track Id,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,!,13,2019-08-09,Trippie Redd,https://open.spotify.com/track/5a1ofLoPiAn5xxf...,United States,top200,NEW_ENTRY,808145.0,5a1ofLoPiAn5xxf5UE6472,...,5,-6.311,1,0.0841,0.02960,0.000000,0.177,0.693,119.961,135339
1,"""42"" - From SR3MM",130,2018-05-04,"Rae Sremmurd, Swae Lee, Slim Jxmmi",https://open.spotify.com/track/5lf91lPnGKtIqsg...,United States,top200,NEW_ENTRY,297731.0,5lf91lPnGKtIqsgmG1z8Ip,...,1,-6.744,1,0.1250,0.00275,0.000000,0.108,0.324,130.039,238020
2,#PROUDCATOWNERREMIX,89,2019-08-23,"XXXTENTACION, Rico Nasty",https://open.spotify.com/track/5dkZ2DrcPJrqwlR...,United States,top200,NEW_ENTRY,361205.0,5dkZ2DrcPJrqwlRQe6Q35e,...,1,-6.229,1,0.3900,0.02870,0.000041,0.127,0.235,145.127,157293
3,$$$ - with Matt Ox,147,2018-04-01,XXXTENTACION,https://open.spotify.com/track/65u1dHQyQyE4y4a...,United States,top200,MOVE_UP,198304.0,65u1dHQyQyE4y4aN2eDmMF,...,1,-6.952,0,0.0650,0.06360,0.000000,0.138,0.508,140.067,130403
4,'Til You Can't,153,2021-10-08,Cody Johnson,https://open.spotify.com/track/4k3lPl8YTKuY8c1...,United States,top200,NEW_ENTRY,249850.0,4k3lPl8YTKuY8c1HelVnm3,...,1,-4.865,1,0.0436,0.05130,0.000000,0.106,0.460,160.087,224213


In [28]:
#get list of keys
# Get a list of all unique values in the 'key' column
key_list = top200['key'].unique()

# Display the list of unique values
print(key_list)

['F' 'C#' 'A' 'D' 'F#' 'G#' 'B' 'D#' 'C' 'A#' 'G' 'E']


In [29]:
#Convert key to Alpha
key_letters = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
top200['key'] = top200['key'].astype(int).apply(lambda x: key_letters[x % 12] if x != -1 else 'Unknown Key')



ValueError: invalid literal for int() with base 10: 'F'

In [30]:
import math
# Round up the 'tempo' column to the nearest whole number
top200['tempo'] = top200['tempo'].apply(lambda x: math.ceil(x))
top200.head()

,title,rank,date,artist,url,region,chart,trend,streams,Spotify Track Id,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,!,13,2019-08-09,Trippie Redd,https://open.spotify.com/track/5a1ofLoPiAn5xxf...,United States,top200,NEW_ENTRY,808145.0,5a1ofLoPiAn5xxf5UE6472,...,F,-6.311,1,0.0841,0.02960,0.000000,0.177,0.693,120,135339
1,"""42"" - From SR3MM",130,2018-05-04,"Rae Sremmurd, Swae Lee, Slim Jxmmi",https://open.spotify.com/track/5lf91lPnGKtIqsg...,United States,top200,NEW_ENTRY,297731.0,5lf91lPnGKtIqsgmG1z8Ip,...,C#,-6.744,1,0.1250,0.00275,0.000000,0.108,0.324,131,238020
2,#PROUDCATOWNERREMIX,89,2019-08-23,"XXXTENTACION, Rico Nasty",https://open.spotify.com/track/5dkZ2DrcPJrqwlR...,United States,top200,NEW_ENTRY,361205.0,5dkZ2DrcPJrqwlRQe6Q35e,...,C#,-6.229,1,0.3900,0.02870,0.000041,0.127,0.235,146,157293
3,$$$ - with Matt Ox,147,2018-04-01,XXXTENTACION,https://open.spotify.com/track/65u1dHQyQyE4y4a...,United States,top200,MOVE_UP,198304.0,65u1dHQyQyE4y4aN2eDmMF,...,C#,-6.952,0,0.0650,0.06360,0.000000,0.138,0.508,141,130403
4,'Til You Can't,153,2021-10-08,Cody Johnson,https://open.spotify.com/track/4k3lPl8YTKuY8c1...,United States,top200,NEW_ENTRY,249850.0,4k3lPl8YTKuY8c1HelVnm3,...,C#,-4.865,1,0.0436,0.05130,0.000000,0.106,0.460,161,224213


In [32]:
#Convert Duration
# convert milliseconds to minutes
top200['duration_mins'] = top200['duration_ms'] / 60000
top200['duration_mins']=top200['duration_mins'].round(2)

# drop old column
top200=top200.drop('duration_ms',axis=1)
top200.head()

,title,rank,date,artist,url,region,chart,trend,streams,Spotify Track Id,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins
0,!,13,2019-08-09,Trippie Redd,https://open.spotify.com/track/5a1ofLoPiAn5xxf...,United States,top200,NEW_ENTRY,808145.0,5a1ofLoPiAn5xxf5UE6472,...,F,-6.311,1,0.0841,0.02960,0.000000,0.177,0.693,120,2.26
1,"""42"" - From SR3MM",130,2018-05-04,"Rae Sremmurd, Swae Lee, Slim Jxmmi",https://open.spotify.com/track/5lf91lPnGKtIqsg...,United States,top200,NEW_ENTRY,297731.0,5lf91lPnGKtIqsgmG1z8Ip,...,C#,-6.744,1,0.1250,0.00275,0.000000,0.108,0.324,131,3.97
2,#PROUDCATOWNERREMIX,89,2019-08-23,"XXXTENTACION, Rico Nasty",https://open.spotify.com/track/5dkZ2DrcPJrqwlR...,United States,top200,NEW_ENTRY,361205.0,5dkZ2DrcPJrqwlRQe6Q35e,...,C#,-6.229,1,0.3900,0.02870,0.000041,0.127,0.235,146,2.62
3,$$$ - with Matt Ox,147,2018-04-01,XXXTENTACION,https://open.spotify.com/track/65u1dHQyQyE4y4a...,United States,top200,MOVE_UP,198304.0,65u1dHQyQyE4y4aN2eDmMF,...,C#,-6.952,0,0.0650,0.06360,0.000000,0.138,0.508,141,2.17
4,'Til You Can't,153,2021-10-08,Cody Johnson,https://open.spotify.com/track/4k3lPl8YTKuY8c1...,United States,top200,NEW_ENTRY,249850.0,4k3lPl8YTKuY8c1HelVnm3,...,C#,-4.865,1,0.0436,0.05130,0.000000,0.106,0.460,161,3.74


In [33]:
# Extract the year and month into separate columns
top200['year'] = top200['date'].dt.year
top200['month'] = top200['date'].dt.month

# Drop the original date column if desired
top200.drop(columns=['date'], inplace=True)

top200.head()

,title,rank,artist,url,region,chart,trend,streams,Spotify Track Id,danceability,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,year,month
0,!,13,Trippie Redd,https://open.spotify.com/track/5a1ofLoPiAn5xxf...,United States,top200,NEW_ENTRY,808145.0,5a1ofLoPiAn5xxf5UE6472,0.725,...,1,0.0841,0.02960,0.000000,0.177,0.693,120,2.26,2019,8
1,"""42"" - From SR3MM",130,"Rae Sremmurd, Swae Lee, Slim Jxmmi",https://open.spotify.com/track/5lf91lPnGKtIqsg...,United States,top200,NEW_ENTRY,297731.0,5lf91lPnGKtIqsgmG1z8Ip,0.968,...,1,0.1250,0.00275,0.000000,0.108,0.324,131,3.97,2018,5
2,#PROUDCATOWNERREMIX,89,"XXXTENTACION, Rico Nasty",https://open.spotify.com/track/5dkZ2DrcPJrqwlR...,United States,top200,NEW_ENTRY,361205.0,5dkZ2DrcPJrqwlRQe6Q35e,0.783,...,1,0.3900,0.02870,0.000041,0.127,0.235,146,2.62,2019,8
3,$$$ - with Matt Ox,147,XXXTENTACION,https://open.spotify.com/track/65u1dHQyQyE4y4a...,United States,top200,MOVE_UP,198304.0,65u1dHQyQyE4y4aN2eDmMF,0.774,...,0,0.0650,0.06360,0.000000,0.138,0.508,141,2.17,2018,4
4,'Til You Can't,153,Cody Johnson,https://open.spotify.com/track/4k3lPl8YTKuY8c1...,United States,top200,NEW_ENTRY,249850.0,4k3lPl8YTKuY8c1HelVnm3,0.501,...,1,0.0436,0.05130,0.000000,0.106,0.460,161,3.74,2021,10


In [34]:
top200.to_csv("Output/SpotifyTop200_Final.csv")